##To activate Elena's Environment conda activate D:/Elena/Cuarto/VC/Enviroments
 

Training the model to detec license plates

from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")

# Train the model
train_results = model.train(
    data="dataset.yaml",  # path to dataset YAML
    epochs=10,  # number of training epochs
    imgsz=640,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Evaluate model performance on the validation set
metrics = model.val()

# Perform object detection on an image
results = model("path/to/image.jpg")
results[0].show()

# Export the model to ONNX format
# path = model.export(format="onnx")  # return path to exported model

In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolo11n.yaml")  # build a new model from YAML
# model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model_objects = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model
res_obj = model_objects.train(data="/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/dataset.yaml", epochs=50, imgsz=640, batch=4)

Detection and Counting of People and Cars


In [2]:
import cv2
from ultralytics import YOLO
import math


model_objects = YOLO('yolo11n.pt')
video_path = 'test-vid.mp4' 

# Etiquetas de las distintas clases
classNames = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
    "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
    "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush"
]

# Inicializar contadores de flujo
flow_counts = {
    'left_to_right': 0,
    'right_to_left': 0
}

# Diccionario para almacenar la posición anterior de los objetos y seguimiento en cada cuadro
tracked_objects = {}

# Cargar el video

vid = cv2.VideoCapture(video_path)

while True:
    # Leer fotograma
    ret, img = vid.read()

    # Finalizar si no hay fotograma
    if not ret:
        break

    # Detectar objetos en el fotograma
    res_obj = model_objects(img, stream=True)

    # Procesar cada detección
    for r in res_obj:
        boxes = r.boxes

        for box in boxes:
            # Coordenadas y clase del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            # Crear un ID único basado en la clase y una clave para el seguimiento
            object_id = f"{detected_class}_{x1}_{y1}_{x2}_{y2}"

            # Verificar si el objeto es una persona o un coche
            if detected_class in ["person", "car"]:
                # Actualizar o añadir la posición del objeto en el diccionario de seguimiento
                if object_id in tracked_objects:
                    # Obtener posición previa del objeto
                    prev_x1, prev_y1, prev_x2, prev_y2 = tracked_objects[object_id]

                    # Calcular dirección del movimiento
                    if x1 > prev_x1:  # Moviendo de izquierda a derecha
                        flow_counts['left_to_right'] += 1
                        # Eliminamos del diccionario para no contar repetidamente
                        del tracked_objects[object_id]
                    elif x1 < prev_x1:  # Moviendo de derecha a izquierda
                        flow_counts['right_to_left'] += 1
                        # Eliminamos del diccionario para no contar repetidamente
                        del tracked_objects[object_id]

                # Almacenar la posición actual del objeto
                tracked_objects[object_id] = (x1, y1, x2, y2)

                # Dibujar el contorno y clase en el fotograma
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, detected_class, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Mostrar los contadores de flujo en el fotograma
    cv2.putText(img, f'Left to Right: {flow_counts["left_to_right"]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(img, f'Right to Left: {flow_counts["right_to_left"]}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Mostrar el fotograma procesado
    cv2.imshow('Video', img)

    # Salir con la tecla ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar el objeto de captura y cerrar ventanas
vid.release()
cv2.destroyAllWindows()



0: 384x640 10 cars, 80.7ms
Speed: 3.0ms preprocess, 80.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 98.2ms
Speed: 3.9ms preprocess, 98.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 92.0ms
Speed: 7.3ms preprocess, 92.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 81.6ms
Speed: 3.6ms preprocess, 81.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 80.9ms
Speed: 2.6ms preprocess, 80.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 84.1ms
Speed: 3.3ms preprocess, 84.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 97.8ms
Speed: 2.5ms preprocess, 97.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 86.7ms
Speed: 2.7ms preprocess, 86.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 3

Making cars and people anonymous 

In [6]:
from ultralytics import YOLO
import cv2
import math



# Carga del modelo
model_objects = YOLO("yolo11n.pt")
model_lipl = YOLO("./datasets/selfmadeYOLO/runs/detect/50-epochs/weights/best.pt")
video_path = 'test-vid.mp4'  
output_path = 'anonymized_output.mp4'


# Etiqueta de las distintas clases
classNames = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
    "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
    "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush", "license plate"  # Añadir 'license plate' aquí
]

# Captura desde el archivo de video
vid = cv2.VideoCapture(video_path)

# Configuración de salida de video anonimizado
fps = int(vid.get(cv2.CAP_PROP_FPS))
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while True:
    ret, img = vid.read()
    
    if not ret:
        break

    # Detecta en la imagen
    res_obj = model_objects.track(img)
    res_lipl = model_lipl.track(img)

    # annotate frame
    img = res_obj[0].plot() 
    img = res_lipl[0].plot(img=img)

    # apply object detection
    for r in res_obj:
        boxes = r.boxes

        for box in boxes:
            # Obtén las coordenadas del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros

            # Obtén la clase detectada
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            # Aplicar desenfoque en áreas específicas
            if detected_class == "person":
                # Aplicar desenfoque en el tercio superior de la persona (cara)
                face_region = img[y1:y1 + (y2 - y1) // 3, x1:x2]
                blurred_face = cv2.GaussianBlur(face_region, (51, 51), 30)
                img[y1:y1 + (y2 - y1) // 3, x1:x2] = blurred_face  # Reemplazar con área desenfocada

    # apply license plate model
    for r in res_lipl: 

        boxes = r.boxes
        for box in boxes:
            # Obtén las coordenadas del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convertir a enteros

            # Obtén la clase detectada
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            #if detected_class == "license plate":
                # Aplicar desenfoque en la región de la matrícula
            plate_region = img[y1:y2, x1:x2]
            blurred_plate = cv2.GaussianBlur(plate_region, (51, 51), 30)
            img[y1:y2, x1:x2] = blurred_plate  # Reemplazar con área desenfocada
            # annotate frame
            #img = res_lipl[0].plot() 
            #r.show()
        

    # Escribe el fotograma anonimizado en el archivo de salida
    out.write(img)

    # Opcional: muestra el video en tiempo real
    cv2.imshow('privacy', img)

    # Salir con ESC
    if cv2.waitKey(20) == 27:
        break

# Libera recursos
vid.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 6 cars, 93.4ms
Speed: 5.1ms preprocess, 93.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 license plates, 78.0ms
Speed: 2.4ms preprocess, 78.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 103.7ms
Speed: 3.4ms preprocess, 103.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 license plates, 87.0ms
Speed: 2.9ms preprocess, 87.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 127.6ms
Speed: 2.9ms preprocess, 127.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 license plates, 77.8ms
Speed: 3.6ms preprocess, 77.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 90.2ms
Speed: 2.9ms preprocess, 90.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 license plates, 76.7ms
Speed: 3.4ms preprocess, 76.7ms inference, 1.0ms postprocess 